%%<br>
mports

In [ ]:
import os, sys, json, re  # Provides OS-dependent functionality, system-specific parameters, JSON handling
import pandas as pd             # Provides data structures and data analysis tools
import numpy as np              # Supports large, multi-dimensional arrays and matrices
import requests
import time
import xlsxwriter
from tqdm import tqdm
from datetime import date #date/time manipulation
import lxml

In [ ]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None  # default='warn'
from IPython.display import display_markdown

Import requests for fetching the web page

In [ ]:
import requests

Import BeautifulSoup for parsing HTML

In [ ]:
from bs4 import BeautifulSoup,SoupStrainer

In [ ]:
from io import StringIO

%%

In [ ]:
soup_url = r'https://ballotpedia.org/State_Legislative_Districts'
# %%

In [ ]:
all_districts = []
response = requests.get(soup_url, verify = False)
soup = BeautifulSoup(response.content, 'html.parser')
state_districts = soup.find_all("a", href = True)

In [ ]:
h_refs = []
for url in state_districts:
    if 'state legislative districts' in str(url):
        # print(url)
        base = "https://ballotpedia.org/"
        full_url = base + str(url.text).replace(' ',"_")
        h_refs.append(full_url)

print(*h_refs, sep="\n")

In [ ]:
for ref in h_refs:
    url = ref
    
    page = requests.get(url)

In [ ]:
    
    os.chdir(r'C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\txt files for troubleshooting')

    # Write the page's text content to a file
    # with open('output_soup_strainer.txt', "w", encoding="utf-8") as f:
    #     f.write(page.text)
    # print(page.content)
    table_strainer = SoupStrainer('table', id='officeholder-table')
    page_soup = BeautifulSoup(page.content, 'html.parser', parse_only=table_strainer)

    # print(page_soup.content)
    # print(type(page_soup))
    districts = page_soup.find_all("a")
    total_districts = []
    # print(page_soup.prettify())
    for d in districts:
        total_districts.append(d.text)
        # print(d.text)
    all_districts.extend(total_districts)

%%

Setting up regex patterns to look for states in other columns

ull state names

In [ ]:
state_list = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", 
    "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", 
    "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", 
    "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", 
    "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", 
    "New Hampshire", "New Jersey", "New Mexico", "New York", 
    "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", 
    "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", 
    "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", 
    "West Virginia", "Wisconsin", "Wyoming", "District of Columbia"
]

tate abbreviations

In [ ]:
state_abbreviations = [
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", 
    "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
    "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
    "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
    "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY", "DC"
]

reation of specific regex to look for state at the beginning of the string

In [ ]:
state_abbreviations_reg = []
for abv in state_abbreviations:
    for_regex = f'^{abv}'
    state_abbreviations_reg.append(for_regex)

ompiling regex patterns for looking for states

In [ ]:
state_pat = re.compile("|".join(state_list))
state_abv_pat = re.compile("|".join(state_abbreviations_reg))
#dictionary creation for future reference in later cells
state_ref = dict(zip(state_list, state_abbreviations))

In [ ]:
codes = list(range(10,61,1))

In [ ]:
state_coding = dict(zip(state_abbreviations, codes))

In [ ]:
thi_states = ['ND', 'NM', 'OH', 'OK', 'VA', 'WV', 'AL', 'CT', 'IL', 'IN', 'KS', 'MO', 'NC']

for k,v in state_coding.items():<br>
    print(k)<br>
    print(v)

%%

In [ ]:
state_intitals = []
for state in all_districts:
    state_match = re.findall(state_pat, state)[0]
    ref_lookup = state_ref.get(str(state_match))
    state_intitals.append(ref_lookup)

In [ ]:
districts_w_intials = pd.DataFrame({'state_abbreviation': state_intitals,'district_string': all_districts})
thi_state_districts = districts_w_intials[districts_w_intials['state_abbreviation'].isin(thi_states)]
thi_state_districts.reset_index(inplace=True, drop=True)

%%splitting up district numbers from rest of string

In [ ]:
lengths = []
thi_state_districts['state_code'] = np.nan
thi_state_districts['chamber_code'] = np.nan
thi_state_districts['district'] = np.nan
thi_state_districts['primary_key'] = np.nan
for i,j in enumerate(thi_state_districts['district_string']):
    # print(str(j))
    # print(row)
    district_raw = re.split(r'\s(?=District)', str(j))
    match = re.findall(r'\s\d+', str(district_raw))[0]
    match = match.strip()
    if len(match) == 2:
        district_code = '0' + str(match)
    elif len(match) == 1:
        district_code = '00'+str(match)
    else:
        district_code = str(match)
    district_len = len(match)
    lengths.append(district_len)
    ext_state = thi_state_districts.loc[i,'state_abbreviation']
    state_code = state_coding.get(ext_state)

In [ ]:
    if 'house' in str(j).lower():
        chamber_code = '0'
    elif 'senate' in str(j).lower():
        chamber_code = '1'
    else:
        print(f'unknown chamber: {str(j)}')
        break
    
    # display_markdown(f'#### {ext_state} - {chamber_code} - {district_raw}', raw=True)
    key_code = f'{state_code}{chamber_code}{district_code}'
    
    
    thi_state_districts.loc[i,'state_code'] = state_code
    thi_state_districts.loc[i,'chamber_code'] = chamber_code
    thi_state_districts.loc[i,'district'] = match
    thi_state_districts.loc[i,'primary_key'] = key_code

In [ ]:
    
    
# %% takes duplicates of primary keys and assigns a seat num

eat num of 00 means there is only one seat (no multi-member districts)

In [ ]:
thi_state_districts['count'] = thi_state_districts['primary_key'].map(thi_state_districts['primary_key'].value_counts())
thi_state_districts['seat_num'] = thi_state_districts.groupby('primary_key').cumcount() + 1
thi_state_districts.loc[thi_state_districts['count'] == 1, 'seat_num'] = 0
thi_state_districts = thi_state_districts.drop(columns='count')
thi_state_districts['seat_num'] = thi_state_districts['seat_num'].apply(lambda x: f'{x:02d}')

In [ ]:
thi_state_districts['final_primary_key'] = thi_state_districts['primary_key'] + thi_state_districts['seat_num']
# %%

In [ ]:
duplicate = thi_state_districts[thi_state_districts.duplicated('primary_key')]

    # print(str(j))
    # print(district_len)
    

print(max(lengths))

In [ ]:
    # print(line.split('District')[-1])

%%

In [ ]:
print(*all_districts, sep='\n')

In [ ]:
[print(x) for x in thi_state_districts]

    # states.append(str(state_match))
# %%

In [ ]:
states = sorted(list(set(states)))
print(len(states))
print(*states, sep="\n")

In [ ]:
for i,j in enumerate(states):
    code = str(i+1)
    if len(code) == 1:
        code = "0"+code
    df = pd.DataFrame({'state': [j], 'code':[code]})
    print(df.to_string())

%%

In [ ]:
    
    break
    print(page.status_code)
    print(page.content[:500])  # Preview the content
    
    # df_list = pd.read_html(page_soup.prettify())
    # print(len(df_list))
    # print(type(df_list))

In [ ]:
    # print(df)

    # dis_soup = BeautifulSoup(response.content, 'html.parser')
    # ditricts_tags = dis_soup.find_all("td")
    # for d in ditricts_tags:
    #     print(d)
    
	

%%